In [1]:
import pandas as pd
import hashlib
from urllib import parse, request
from datetime import date, timedelta
from time import strftime, gmtime
import pandas as pd
from pandas import DataFrame
from io import StringIO

from pandasql import sqldf
import pyodbc
import time

In [12]:
my_merchant_id = 'XXXXX'
api_token = 'XXXXXXXXXXXXXXXXXXX'
api_secret_key = 'XXXXXXXXXXXXXXXXXXXXXXXXXXX'
my_timestamp = strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime())
api_version = 3.0
action_verb = 'transactiondetail'

date_start = date.today() - timedelta(days = 64)
date_end = date.today() - timedelta(days = 1)
export_type = 'pipe'
sortcol = 'transid'
sortdir = 'desc'

In [13]:
# params
data = parse.urlencode({'merchantId': my_merchant_id, 'token': api_token, 
						'version': api_version, 'action': action_verb, 
						'sortCol' : sortcol, 'sortDir' : sortdir, 
						'dateStart' : date_start, 'dateEnd' : date_end, 
						'format': export_type})

# authentication
sig = api_token + ':' + my_timestamp + ':' + action_verb + ':' + api_secret_key
sig_hash = hashlib.sha256(sig.encode('utf-8')).hexdigest()
my_headers = {'x-ShareASale-Date': my_timestamp,
				'x-ShareASale-Authentication': sig_hash}

In [14]:
call = request.Request('https://shareasale.com/w.cfm?%s' % data, headers=my_headers)

try:
	response = request.urlopen(call).read()
except Exception as inst:
	print(inst)

In [15]:
# decode
output = response.decode("utf-8")

# push into df
df = pd.read_csv(StringIO(output), sep='|', engine= 'python', 
usecols = ['transID','userID','transdate','transamount','commission','ssamount','comment','voided','locked',
        'pending','lastip','lastreferer','bannernumber','bannerpage','dateoftrans','dateofclick','timeofclick',
        'dateofreversal','returndays','toolID','storeID','lockDate','transactionType','CommissionType','skulist',
        'priceList','quantityList','orderNumber','parentTrans','bannerName','bannerType','couponCode','referenceTrans',
        'newCustomerFlag','userAgent','originalCurrency','originalCurrencyAmount','isMobile','usedACoupon','merchantDefinedType'])

# Transaction Volume Check        
df.shape

(3256, 40)

In [17]:
# PandaSQL queries. Easier to share/explain steps.
pysqldf = lambda q: sqldf(q, globals())

# For ODBC driver.
drvr = '{ODBC Driver 17 for SQL Server}'
srvr = 'XXXXXXXX'
db = 'KP'
usr ='XXXXXXX'
pw = 'XXXXXXXXX'

# Init ODBC connection.
connection = pyodbc.connect(DRIVER=drvr, SERVER=srvr, DATABASE=db, UID=usr, PWD=pw, Trusted_Connection='YES')

In [ ]:
# Create df for "New" Customers according to Share-A-Sale.
# Looking commissions not yet voided, and not yet locked for payment.
newCust = pysqldf(
"SELECT ORDERNUMBER "
"FROM df "
"WHERE NEWCUSTOMERFLAG = 1 "    # New customer flag.
"AND VOIDED IS NULL "           # Is either null or 1.
"AND LOCKED IS NULL "           # Is either null or 1.
"AND STOREID IN ('1','2')"      # WC/KP ID's.
)

# handle float64
newCust['orderNumber']  = newCust['orderNumber'].astype(int)

# Create df for all order numbers.
orderNos = pysqldf(
"SELECT ORDERNUMBER "
"FROM df "
"WHERE VOIDED IS NULL "             # Is either null or 1.
"AND LOCKED IS NULL "               # Is either null or 1.
"AND STOREID IN ('1','2')"          # WC/KP ID's.
)

# handle float64
orderNos['orderNumber']  = orderNos['orderNumber'].astype(int)

In [18]:
### NEW CUSTOMER REVIEW ###

# Validate Share-A-Sale's "new" customers against response server.
# Find assocaited customer IDs with SAS order numbers, compare order date to their first transaction date.
sql = (
    "SELECT A.ORDER_NO, A.WEB_ORDER_ID, A.ORDER_DATE, B.FIRST_PURCH "               
    "FROM KP.RESPONSE.SYSOENT A "                                                   # Order table.
    "INNER JOIN KP.RESPONSE.SYSCUST B "                                             # Customer table.
	"ON A.CUSTOMER_ID = B.CUSTOMER_ID "
    "WHERE CAST(A.ORDER_DATE AS DATE) > CAST(B.FIRST_PURCH AS DATE) "               # Where order date > Cust_ID's first purchase.
    "AND WEB_ORDER_ID IN {}".format(tuple(str(x) for x in newCust['orderNumber']))  # Dynamicaly pass string of order numbers.
)

# ODBC call.
cursor = connection.cursor()
response_newCust = pd.read_sql(sql, connection)

# List of orders to reduce commission rates.
commissionList = pysqldf(
"SELECT A.WEB_ORDER_ID, A.ORDER_DATE "
"FROM response_newCust "
)

# Order Volume Check  
len(commissionList)

75

In [1]:
### ORDER CONTENTS REVIEW ###

# Commissions on orders that contain certain products are to be voided, per contract.
# Looking items in the product class 'PATTM' or if they contain a giftcard 'GIFT'.
sql2 = (
    "SELECT A.ORDER_NO, A.WEB_ORDER_ID, A.ORDER_DATE "
    "FROM KP.RESPONSE.SYSOENT A "                                                       # Order level table.              
    "WHERE ORDER_NO IN ( "
    "SELECT DISTINCT ORDER_NO"                                                          # Subqeury for order numbers.                                  
    "FROM KP.RESPONSE.LINEITEM A "                                                      # Line level order table.
    "INNER JOIN KP.RESPONSE.INVENTOR B "                                                # Inventory details table.
    "ON A.ITEM_ID = B.ITEM_ID "
    "WHERE (B.PROD_CLASS_ID = 'PATTM' OR A.ITEM_ID = 'GIFT') "                          # Looking for PATTM or GIFT.
    "AND A.ORDER_DATE > GETDATE() - 64 )"                                               # Max lock window. 
    "AND A.WEB_ORDER_ID IN {}".format(tuple(str(x) for x in orderNos['orderNumber']))   # Dynamicaly pass string of SAS order numbers.
)

cursor = connection.cursor()
response_itemID = pd.read_sql(sql2, connection)

# List of orders to void.
itemIDList = pysqldf(
"SELECT WEB_ORDER_ID, ORDER_DATE "
"FROM response_itemID "
)

# Order Volume Check  
len(itemIDList)

NameError: name 'orderNos' is not defined

In [20]:
### ORDER STATUS REVIEW ###

# Looking for orders that have been canceled internally, but still readied for payout with a 'NULL' void SAS status.
sql3 = (
    "SELECT A.ORDER_NO, A.WEB_ORDER_ID, A.ORDER_DATE, A.ORDER_STATUS "                      
    "FROM KP.RESPONSE.SYSOENT A "                                                           # Order table.
    "WHERE A.ORDER_STATUS = 'X' "                                                           # Canceled order status.
    "AND A.WEB_ORDER_ID IN {}".format(tuple(str(x) for x in orderNos['orderNumber']))       # Dynamicaly pass string of SAS order numbers.
)

# ODBC call.
cursor = connection.cursor()
response_canceled = pd.read_sql(sql3, connection)

# List of orders to void.
canceledList = pysqldf(
"SELECT WEB_ORDER_ID, ORDER_DATE "
"FROM response_canceled"
)

# Order Volume Check 
len(canceledList)

0

In [27]:
def voidAPI(ordernumber, date):
    data = parse.urlencode({'merchantId': my_merchant_id, 'token': api_token, 'version': api_version, 'action': 'void', 'ordernumber': ordernumber, 'date': date, 'reason': 'Non-Fulfilled Requirements'})
    sig = api_token + ':' + my_timestamp + ':' + 'void' + ':' + api_secret_key
    sig_hash = hashlib.sha256(sig.encode('utf-8')).hexdigest()
    my_headers = {'x-ShareASale-Date': my_timestamp, 'x-ShareASale-Authentication': sig_hash}
    call = request.Request('https://shareasale.com/w.cfm?%s' % data, headers=my_headers)
    response = request.urlopen(call)
    return response


def editAPI(ordernumber, date):
    data = parse.urlencode({'merchantId': my_merchant_id, 'token': api_token, 'version': api_version, 'action': 'edit', 'ordernumber': ordernumber, 'date': date, 'newpercentage': '10.00', 'newcomment': 'Edit: Non-new customer. Commission rate reduced.'})
    sig = api_token + ':' + my_timestamp + ':' + 'edit' + ':' + api_secret_key
    sig_hash = hashlib.sha256(sig.encode('utf-8')).hexdigest()
    my_headers = {'x-ShareASale-Date': my_timestamp, 'x-ShareASale-Authentication': sig_hash}
    call = request.Request('https://shareasale.com/w.cfm?%s' % data, headers=my_headers)
    response = request.urlopen(call)
    return response

In [21]:
# Void orders containing ineligible items. 
for index, row in itemIDList.iterrows():
    voidAPI(row['WEB_ORDER_ID'], row['ORDER_DATE'])
    time.sleep(5) # Throws errors if too many pass to quickly.

In [32]:
# Reduce commission rates from 30% to 10% with validated non-new customer.
for index, row in commissionList.iterrows():
    editAPI(row['WEB_ORDER_ID'], row['ORDER_DATE'])
    time.sleep(5) # Throws errors if too many pass to quickly.

In [ ]:
# Void commissions for canceled orders.
for index, row in canceledList.iterrows():
    voidAPI(row['WEB_ORDER_ID'], row['ORDER_DATE'])
    time.sleep(5) # Throws errors if too many pass to quickly.